# Handwritten numbers Recognition

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras 

## Load Data

Extraer datos. Las imágenes son de 28*28 píxeles, por lo tanto el csv tiene 784 columnas más 1 indicando la salida

In [16]:
def extract():
    df_train = pd.read_csv('data/mnist_train.csv', delimiter = ',').to_numpy()
    df_test = pd.read_csv('data/mnist_test.csv', delimiter = ',').to_numpy()
    df = pd.DataFrame(np.concatenate((df_train,df_test)))
    return df

Transformar output a array de 10 y separar de las inputs

In [13]:
def transform(df): 
    y_raw = df.iloc[:, 0].to_numpy()
    x = df.drop(df.columns[0], axis='columns').to_numpy()
    y = []
    for i in range(0,y_raw.shape[0]):
        row=np.zeros(10)
        row[y_raw[i]]=1
        y.append(row)
    y=np.array(y)
    y.reshape(y.shape[0],10)
    return x, y

Creamos los dataset de entrenamiento y test

In [17]:
df = extract()
x_train,y_train = transform(df.iloc[0:60000])
#x_val,y_val = transform(df.iloc[50000:60000])
x_test,y_test = transform(df.iloc[60000:70000])

In [20]:
x_train = x_train.reshape(60000,28,28,1)
x_test = x_test.reshape(9998,28,28,1)

## Creación de la Red Neuronal

En la red neuronal vamoas a implementar una arquitectura de (784,512,256,128,10), con funciones de activación ReLu en todas las capas menos en la última, en la cual utilizaremos la funcion softmax (https://www.quora.com/Why-is-it-better-to-use-Softmax-function-than-sigmoid-function).<br> Para cada capa se utilizará la funcion Dense, la cual tiene los siguientes parámetros por defecto:
- activation=None,
- use_bias=True,
- kernel_initializer='glorot_uniform',
- bias_initializer='zeros',
- kernel_regularizer=None,
- bias_regularizer=None,
- activity_regularizer=None,
- kernel_constraint=None,
- bias_constraint=None




In [23]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

Using TensorFlow backend.


In [24]:
model = Sequential()

#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

Vamos a crear y entrenar el modelo, 50 epochs serán suficientes.

In [26]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#learning_rate=0.001
model.fit(x_train, y_train,validation_data=(x_test, y_test), epochs=5)

Train on 60000 samples, validate on 9998 samples
Epoch 1/5
60000/60000 [==============================] - 80s 1ms/step - loss: 0.2252 - accuracy: 0.9515 - val_loss: 0.0898 - val_accuracy: 0.9717
Epoch 2/5
60000/60000 [==============================] - 81s 1ms/step - loss: 0.0680 - accuracy: 0.9793 - val_loss: 0.1020 - val_accuracy: 0.9729
Epoch 3/5
60000/60000 [==============================] - 81s 1ms/step - loss: 0.0479 - accuracy: 0.9850 - val_loss: 0.0888 - val_accuracy: 0.9733
Epoch 4/5
60000/60000 [==============================] - 82s 1ms/step - loss: 0.0361 - accuracy: 0.9885 - val_loss: 0.1055 - val_accuracy: 0.9734
Epoch 5/5
60000/60000 [==============================] - 83s 1ms/step - loss: 0.0273 - accuracy: 0.9912 - val_loss: 0.1120 - val_accuracy: 0.9776


Evaluamos el modelo con el acierto de los datos de prueba

In [27]:
tes_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Acc: ', test_acc)

9998/9998 [==============================] - 2s 213us/step
Test Acc:  0.9775955080986023


## Regularización

In [16]:
from keras import regularizers
modelR = keras.Sequential([
    keras.layers.Dense(784, input_dim=784, kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(512, activation="relu", kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(10, activation="softmax", kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01))
])

Using TensorFlow backend.


In [17]:
modelR.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#learning_rate=0.001
modelR.fit(x_train, y_train, epochs = 50, batch_size=200, verbose=1)

Train on 60000 samples
Epoch 1/50
60000/60000 [==============================] - 4s 70us/sample - loss: 17.9050 - accuracy: 0.8753
Epoch 2/50
60000/60000 [==============================] - 4s 62us/sample - loss: 10.8157 - accuracy: 0.9478
Epoch 3/50
60000/60000 [==============================] - 4s 63us/sample - loss: 7.6734 - accuracy: 0.9663
Epoch 4/50
60000/60000 [==============================] - 4s 63us/sample - loss: 5.4694 - accuracy: 0.9738
Epoch 5/50
60000/60000 [==============================] - 5s 79us/sample - loss: 3.9047 - accuracy: 0.9774
Epoch 6/50
60000/60000 [==============================] - ETA: 0s - loss: 2.8039 - accuracy: 0.97 - 5s 81us/sample - loss: 2.7996 - accuracy: 0.9777
Epoch 7/50
60000/60000 [==============================] - 5s 81us/sample - loss: 2.0298 - accuracy: 0.9759
Epoch 8/50
60000/60000 [==============================] - 5s 80us/sample - loss: 1.4890 - accuracy: 0.9756
Epoch 9/50
60000/60000 [==============================] - 5s 80us/sample - lo

In [19]:
tes_loss, test_acc = modelR.evaluate(x_test, y_test)
print('Test Acc: ', test_acc)

9998/9998 [==============================] - 2s 244us/sample - loss: 0.2920 - accuracy: 0.9649
Test Acc:  0.964893
